In [1]:
import pandas as pd
import spacy
import snorkel

# Reading in and working with the data
The few-NERD data is much more detailed than we need, if we want to make a direct comparison to the silver data we have previously worked on. Therefore, we need to read in the few-NERD data and convert it to our preferred format (with labels O-6). We then save that in a new csv file so we don't have to convert everything every time we want to work with the data. In the future, only the already converted csv will be read in ;-)
```
csv_data=pd.read_csv("supervised/train.txt", sep="\t", header=None, names=["Tokens", "Tags"], skip_blank_lines=False)
started=False
for index, row in csv_data.iterrows():
    tag=row.iloc[1]
    
    if pd.isna(csv_data.iloc[index, 1]):
        continue
        
    if tag=="O":
        started=False
    
    elif tag.startswith("art") or tag.startswith("product") or tag.startswith("event"):
        csv_data.iloc[index, 1]="O"
        started=False
        
    elif tag.startswith("building") or tag.startswith("location"):
        if started==False:
            csv_data.iloc[index, 1]="5"
            started=True
            
        else:
            csv_data.iloc[index, 1]="6"
        
    elif tag.startswith("organization"):
        if started==False:
            csv_data.iloc[index, 1]="3"
            started=True
            
        else:
            csv_data.iloc[index, 1]="4"
            
    elif tag.startswith("person"):
        if started==False:
            csv_data.iloc[index, 1]="1"
            started=True
        else:
            csv_data.iloc[index, 1]="2"
            
    elif tag.startswith("other"):
        if tag=="other-god":
            if started==False:
                csv_data.iloc[index, 1]="1"
                started=True
            else:
                csv_data.iloc[index, 1]="2"
        else:
            csv_data.iloc[index, 1]="O"
            started=False
            
    else:
        print("Weird!")
csv_data.to_csv("fewnerd-supervised-training-data.csv", sep="\t", index=False)
```

In [2]:
processed_data=[]
for line in open("fewnerd-supervised-training-data.csv", encoding='utf-8'):
    line=line.strip()
    processed_data.append(line)

In [ ]:
sentence=""
words=[]
lines=[]
for i, tokenandtag in reversed(list(enumerate(processed_data))):
    if tokenandtag == "":
        sentence=" ".join(reversed(words))
        words=[]
        processed_data=processed_data[:i+1]+[f"#", f"# text = {sentence}"]+processed_data[i+1:]
    elif i==0:
        sentence=" ".join(reversed(words))
        word, _ = tokenandtag.split("\t")
        processed_data=[f"# sent_id = {i+1}"]+[f"# text = {word} {sentence}"]+processed_data[i:]
    else:
        try:
            token, tag = tokenandtag.split("\t")
            words.append(token)
        except:
            print(tokenandtag)


In [ ]:
sent_id=1
word_cnt=1
text=False

for i, line in enumerate(processed_data):
    if line=="":
        continue
        
    if text==True:
        text=False
        continue
        
    if line[0]=="#":
        processed_data[i]=f"# sent_id = {sent_id}"
        sent_id+=1
        word_cnt=1
        text=True
        
    else:
        processed_data[i]=f"{word_cnt}\t"+processed_data[i]
        word_cnt+=1

print("\n".join(data[0:100]))

In [ ]:
with open("fewnerd-supervised-training-data.csv", encoding="utf-8") as file:
    train=file.read()
    
training_data_70=pd.read_csv("fewnerd-supervised-training-data.csv", sep="\t")
validation_data_10=pd.read_csv("fewnerd-supervised-validation-data.csv", sep="\t")
testing_data_20=pd.read_csv("fewnerd-supervised-testing-data.csv", sep="\t")

In [ ]:
training_data_70.head(40)

# With SpaCy
SpaCy recommends using their config file through the command line

In [ ]:
from spacy.cli.train import train

train("base_config.cfg", overrides={"paths.train": "./train.spacy", "paths.dev": "./dev.spacy"})


In [ ]:
from spacy.cli import download

download("en_core_web_sm")

In [ ]:
nlp=spacy.load("en_core_web_sm")
doc=nlp(test123)
for ent in doc.ents:
    print(ent.text, " | ", ent.label_)

# With Snorkel
https://github.com/snorkel-team/snorkel

# With Python Spark
https://medium.com/john-snow-labs/named-entity-recognition-ner-with-python-at-scale-a37d7c241135